# Формальные степенные ряды

Каждый ряд $f(x)$ -- это бесконечная последовательность коэффициентов. То есть ряд
$$
f(x) = f_0 + f_1 x + f_2 x^2 + f_3 x^3 + ...
$$
есть последовательность
$$
[f_0, f_1, f_2, f_3, ...].
$$

Последовательность мы представляем как структуру, часть вычислений которой отложенные. Так описанная выше последовательность есть `Cons(f_0, Cons(f_1, Cons(f_2, Cons(f_3, Thunk))))`. Первые четыре элемента вычислены, а для вычисления пятого надо вызвать функцию, которая этот пятый элемент посчитает.

Cons-ячейка -- это пара, которая даёт информацию о том, каково значение текущего коэффициента, и как вычислить значения всех остальных коэффициентов.

In [1]:
import Base: show, +, *, /, -, inv
using Latexify

In [2]:
abstract type Series{T <: Number}
end


mutable struct SeriesThunk{T <: Number}
    expr::Series{T}
end


struct Cons{T <: Number} <: Series{T}
    head::T
    tail::SeriesThunk{T}
end

function force end

function force(t::SeriesThunk{T})::Cons{T} where T <: Number
    return t.expr = force(t.expr)
end

function force(s::Cons{T})::Cons{T} where T <: Number
    return s
end

function consthunk(head::T, tail::Series{T})::Series{T} where T <: Number
    return Cons{T}(head, SeriesThunk{T}(tail))
end

consthunk (generic function with 1 method)

In [3]:
struct SeriesGenerator{T <: Number} <: Series{T}
    index::Int
    generator::Function
end


"Конструктор степенного ряда из генерирующей функции"
function generate(::Type{T}, gen::Function)::Series{T} where T <: Number
    return SeriesGenerator{T}(0, gen)
end


function force(s::SeriesGenerator{T})::Cons{T} where T <: Number
    gen = s.generator
    return consthunk(T(gen(s.index)), SeriesGenerator{T}(s.index + 1, gen))
end


struct SeriesVector{T <: Number} <: Series{T}
    index::Int
    data::Vector{T}
    cycled::Bool
end

"Конструктор степенного ряда из вектора"
function series(data::Vector{T}; cycled::Bool=false)::Series{T} where T <: Number
    return SeriesVector{T}(0, data, cycled)
end

function force(s::SeriesVector{T})::Cons{T} where T <: Number
    n = length(s.data)
    i = s.index
    if s.cycled
        return consthunk(s.data[i % n + 1], SeriesVector{T}(i + 1, s.data, s.cycled))
    else
        return consthunk((i >= n) ? T(0) : s.data[i + 1], SeriesVector{T}(i + 1, s.data, s.cycled))
    end
end

force (generic function with 4 methods)

In [4]:
"Преобразовать часть степенного ряда в выражение"
function series_expr(s::Series{T}, n::Int=9, var::Symbol=:x)::Expr where T <: Number
    cons = force(s)::Cons{T}
    args = Any[cons.head]
    s = cons.tail

    for i in 1:n
        cons = force(s)::Cons{T}
        coef = cons.head
        s = cons.tail

        term = (i == 1) ? var : :($var ^ $i)
        term = (coef == 1) ? term : :($coef * $term)
        push!(args, term)
    end
    return Expr(:call, :+, args..., :...)
end

"Красивое представление степенного ряда в LaTeX"
function show(io::IO, ::MIME"text/latex", s::Series{T})::Nothing where T <: Number
    println(io, latexify(series_expr(s)))
end

show (generic function with 263 methods)

Далее ряды будем представлять следующим образом:
$$
f(x) = f_0 + f_1 x + f_2 x^2 + \ldots = f_h + x \cdot f_t(x),
$$

где $f_h = f_0$ есть "голова"(head), а $f_t(x) = f_1 + f_2 x + f_3 x^2 + \ldots$ есть "хвост"(tail)

### Сложение степенных рядов
Пусть есть ряды $f(x) = f_h + x \cdot f_t(x)$ и $g(x) = g_h + x \cdot g_t(x)$, тогда:

$$
f(x) + g(x) = \left(f_h + g_h\right) + x \cdot \left(f_t(x) + g_t(x)\right)
$$

In [5]:
struct SeriesAdd{T <: Number} <: Series{T}
    first::SeriesThunk{T}
    second::SeriesThunk{T}
end


function +(s1::Series{T}, s2::Series{T})::Series{T} where T <: Number
    return SeriesAdd{T}(SeriesThunk{T}(s1), SeriesThunk{T}(s2))
end


function force(s::SeriesAdd{T})::Cons{T} where T <: Number
    first = force(s.first)
    second = force(s.second)
    return consthunk(first.head + second.head, SeriesAdd{T}(first.tail, second.tail))
end

force (generic function with 5 methods)

### Умножение ряда на число
Пусть есть число $a$ и ряд $f(x) = f_h + x \cdot f_t(x)$, тогда:

$$
a \cdot f(x) = \left(a \cdot f_h \right) + x \cdot \left(a \cdot f_t(x)\right)
$$

In [6]:
struct SeriesScale{T <: Number} <: Series{T}
    scaler::T
    series::SeriesThunk{T}
end


function *(a::T, f::Series{T})::Series{T} where T <: Number
    return SeriesScale{T}(a, SeriesThunk{T}(f))
end


function -(f::Series{T})::Series{T} where T <: Number
    return SeriesScale{T}(T(-1), SeriesThunk{T}(f))
end


function force(s::SeriesScale{T})::Cons{T} where T <: Number
    c = force(s.series)::Cons{T}
    return consthunk(s.scaler * c.head, SeriesScale{T}(s.scaler, c.tail))
end

force (generic function with 6 methods)

### Умножение рядов

Пусть есть ряды $f(x) = f_h + x f_t(x)$ и $g(x)$, тогда:

$$
f(x) \cdot g(x) = \left(f_h \cdot g(x)\right) + x \cdot \left(f_t(x) \cdot g(x)\right)
$$

In [7]:
struct SeriesMul{T <: Number} <: Series{T}
    fst::SeriesThunk{T}
    snd::SeriesThunk{T}
end


function *(f::Series{T}, g::Series{T})::Series{T} where T <: Number
    return SeriesMul{T}(SeriesThunk{T}(f), SeriesThunk{T}(g))
end


function force(s::SeriesMul{T})::Cons{T} where T <: Number
    fst = force(s.fst)
    snd = force(s.snd)
    return force(fst.head * snd + consthunk(T(0), SeriesMul{T}(s.snd, fst.tail)))
end

force (generic function with 7 methods)

### Подстановка
Пусть есть ряды $f(x) = f_h + x f_t(x)$ и $g(x) = x g_t(x)$, тогда:

$$
f(g(x)) = f_h + g(x) \cdot f_t(g(x)) = f_h + x \cdot g_t(x) \cdot f_t(g(x))
$$

Наличие $x$ во втором слагаемом даёт возможность отложить вычисление этого второго слагаемого.

In [8]:
struct SeriesApply{T <: Number} <: Series{T}
    fun::SeriesThunk{T}
    arg::SeriesThunk{T}
end


function (f::Series{T})(g::Series{T})::Series{T} where T <: Number
    return SeriesApply{T}(SeriesThunk{T}(f), SeriesThunk{T}(g))
end


function force(s::SeriesApply{T})::Cons{T} where T <: Number
    arg = force(s.arg)::Cons{T}
    arg.head == 0 || throw(ErrorException("non-zero head in application"))

    fun = force(s.fun)::Cons{T}
    return consthunk(fun.head, SeriesMul{T}(arg.tail, SeriesThunk{T}(SeriesApply{T}(fun.tail, s.arg))))
end

force (generic function with 8 methods)

### Обращение ряда (в смысле деления)

Пусть есть ряд $f(x) = f_h + x \cdot f_t(x)$, тогда:

$$
\frac{1}{f(x)} = \frac{1}{f_h + x \cdot f_t(x)} = \frac{1}{f_h} \cdot \frac{1}{1 + x \frac{f_t(x)}{f_h}}
= \frac{1}{f_h} \cdot r\left(-x \frac{f_t(x)}{f_h}\right),
$$
где $r(x) = 1 + x + x^2 + x^3 + \ldots$

In [9]:
"Обращение ряда (в смысле деления)"
function recip(s::Series{T})::Series{T} where T <: Number
    c = force(s)::Cons{T}
    c.head == 0 && throw(ErrorException("zero head in reciprocation"))

    ones = series(T[1], cycled=true)
    rh = 1 / c.head
    return rh * ones(consthunk(T(0), SeriesScale{T}(-rh, c.tail)))
end

recip

### Деление рядов

$$
\frac{f(x)}{g(x)} = f(x) \cdot \frac{1}{g(x)}
$$

In [10]:
function /(s1::Series{T}, s2::Series{T})::Series{T} where T <: Number
    den = force(s2)::Cons{T}
    if den.head == 0
        num = force(s1)::Cons{T}
        num.head == 0 || throw(ErrorException("zero head in reciprocation"))
        return force(num.tail) / force(den.tail)
    else
        return s1 * recip(den)
    end
end

/ (generic function with 107 methods)

In [11]:
"Решить уравнение f(g(x)) = h(x)"
struct SeriesSolve{T <: Number} <: Series{T}
    arg::SeriesThunk{T}
    res::SeriesThunk{T}
end


function solve(g::Series{T}, h::Series{T}=series(T[0, 1])) where T <: Number
    return SeriesSolve{T}(SeriesThunk{T}(g), SeriesThunk{T}(h))
end


function force(s::SeriesSolve{T})::Cons{T} where T <: Number
    arg = force(s.arg)::Cons{T}
    arg.head == 0 || throw(ErrorException("non-zero head of argument in solve"))
    
    res = force(s.res)::Cons{T}
    return consthunk(res.head, SeriesSolve{T}(s.arg, SeriesThunk{T}(force(res.tail) / force(arg.tail))))
end

force (generic function with 9 methods)

In [12]:
"Обращение ряда (в смысле подстановки)"
function inv(g::Series{T})::Series{T} where T <: Number
    return solve(g)
end

Base.inv

In [13]:
"Поиск фиксированной точки, т.е. решение уравнения s = f(s)"
mutable struct SeriesFix{T <: Number} <: Series{T}
    fun::Union{Function, SeriesThunk{T}}
end


function force(s::SeriesFix{T})::Cons{T} where T <: Number
    if isa(s.fun, Function)
        s.fun = SeriesThunk{T}(s.fun(s))
    end

    return force(s.fun)
end

force (generic function with 10 methods)

In [14]:
struct SeriesDiff{T <: Number} <: Series{T}
    arg::Series{T}
    index::Int
end


"Дифференцирование"
function differentiate(s::Series{T})::Series{T} where T <: Number
    s = force(s).tail
    return SeriesDiff{T}(s, T(1))
end

    
function force(s::SeriesDiff{T})::Cons{T} where T <: Number
    arg = force(s.arg)::Cons{T}
    return consthunk(arg.head * T(s.index), SeriesDiff{T}(arg.tail, s.index + 1))
end

force (generic function with 11 methods)

In [15]:
struct SeriesInt{T <: Number} <: Series{T}
    arg::SeriesThunk{T}
    index::Int
end


"Интегрирование"
function integrate(s::Series{T}, initial::T=T(0))::Series{T} where T <: Number
    return consthunk(initial, SeriesInt{T}(SeriesThunk{T}(s), T(1)))
end


function force(s::SeriesInt{T})::Cons{T} where T <: Number
    arg = force(s.arg)::Cons{T}
    return consthunk(arg.head / T(s.index), SeriesInt{T}(arg.tail, s.index + 1))
end

force (generic function with 12 methods)

In [16]:
ser0 = generate(Rational{Int}, i -> i == 2 ? -1 : 0)

$0 + 0 \cdot x -1 \cdot x^{2} + 0 \cdot x^{3} + 0 \cdot x^{4} + 0 \cdot x^{5} + 0 \cdot x^{6} + 0 \cdot x^{7} + 0 \cdot x^{8} + 0 \cdot x^{9} + ...$


In [17]:
ser1 = generate(Rational{Int}, i -> 1//1)

$1 + x + x^{2} + x^{3} + x^{4} + x^{5} + x^{6} + x^{7} + x^{8} + x^{9} + ...$


In [18]:
ser2 = generate(Rational{Int}, i -> (i == 0) ? 0 : (-1)^i // i)

$0 -1 \cdot x + \frac{1}{2} \cdot x^{2} + \frac{-1}{3} \cdot x^{3} + \frac{1}{4} \cdot x^{4} + \frac{-1}{5} \cdot x^{5} + \frac{1}{6} \cdot x^{6} + \frac{-1}{7} \cdot x^{7} + \frac{1}{8} \cdot x^{8} + \frac{-1}{9} \cdot x^{9} + ...$


In [19]:
force(ser0)

$0 + 0 \cdot x -1 \cdot x^{2} + 0 \cdot x^{3} + 0 \cdot x^{4} + 0 \cdot x^{5} + 0 \cdot x^{6} + 0 \cdot x^{7} + 0 \cdot x^{8} + 0 \cdot x^{9} + ...$


In [20]:
ser1 + ser2

$1 + 0 \cdot x + \frac{3}{2} \cdot x^{2} + \frac{2}{3} \cdot x^{3} + \frac{5}{4} \cdot x^{4} + \frac{4}{5} \cdot x^{5} + \frac{7}{6} \cdot x^{6} + \frac{6}{7} \cdot x^{7} + \frac{9}{8} \cdot x^{8} + \frac{8}{9} \cdot x^{9} + ...$


In [21]:
1//3 * ser1

$\frac{1}{3} + \frac{1}{3} \cdot x + \frac{1}{3} \cdot x^{2} + \frac{1}{3} \cdot x^{3} + \frac{1}{3} \cdot x^{4} + \frac{1}{3} \cdot x^{5} + \frac{1}{3} \cdot x^{6} + \frac{1}{3} \cdot x^{7} + \frac{1}{3} \cdot x^{8} + \frac{1}{3} \cdot x^{9} + ...$


In [22]:
ser2 * ser2

$0 + 0 \cdot x + x^{2} -1 \cdot x^{3} + \frac{11}{12} \cdot x^{4} + \frac{-5}{6} \cdot x^{5} + \frac{137}{180} \cdot x^{6} + \frac{-7}{10} \cdot x^{7} + \frac{363}{560} \cdot x^{8} + \frac{-761}{1260} \cdot x^{9} + ...$


In [23]:
ser0(ser2)

$0 + 0 \cdot x -1 \cdot x^{2} + x^{3} + \frac{-11}{12} \cdot x^{4} + \frac{5}{6} \cdot x^{5} + \frac{-137}{180} \cdot x^{6} + \frac{7}{10} \cdot x^{7} + \frac{-363}{560} \cdot x^{8} + \frac{761}{1260} \cdot x^{9} + ...$


In [24]:
recip(ser1)

$1 -1 \cdot x + 0 \cdot x^{2} + 0 \cdot x^{3} + 0 \cdot x^{4} + 0 \cdot x^{5} + 0 \cdot x^{6} + 0 \cdot x^{7} + 0 \cdot x^{8} + 0 \cdot x^{9} + ...$


In [25]:
ser0 / ser0

$1 + 0 \cdot x + 0 \cdot x^{2} + 0 \cdot x^{3} + 0 \cdot x^{4} + 0 \cdot x^{5} + 0 \cdot x^{6} + 0 \cdot x^{7} + 0 \cdot x^{8} + 0 \cdot x^{9} + ...$


In [26]:
ser1(ser0)

$1 + 0 \cdot x -1 \cdot x^{2} + 0 \cdot x^{3} + x^{4} + 0 \cdot x^{5} -1 \cdot x^{6} + 0 \cdot x^{7} + x^{8} + 0 \cdot x^{9} + ...$


In [27]:
ser2 / ser2

$1 + 0 \cdot x + 0 \cdot x^{2} + 0 \cdot x^{3} + 0 \cdot x^{4} + 0 \cdot x^{5} + 0 \cdot x^{6} + 0 \cdot x^{7} + 0 \cdot x^{8} + 0 \cdot x^{9} + ...$


In [28]:
3//1 * ser1 * ser1

$3 + 6 \cdot x + 9 \cdot x^{2} + 12 \cdot x^{3} + 15 \cdot x^{4} + 18 \cdot x^{5} + 21 \cdot x^{6} + 24 \cdot x^{7} + 27 \cdot x^{8} + 30 \cdot x^{9} + ...$


In [29]:
inv(ser2)

$0 -1 \cdot x + \frac{1}{2} \cdot x^{2} + \frac{-1}{6} \cdot x^{3} + \frac{1}{24} \cdot x^{4} + \frac{-1}{120} \cdot x^{5} + \frac{1}{720} \cdot x^{6} + \frac{-1}{5040} \cdot x^{7} + \frac{1}{40320} \cdot x^{8} + \frac{-1}{362880} \cdot x^{9} + ...$


In [34]:
@time display(inv(inv(inv(inv(inv(inv(ser2)))))))

$0 -1 \cdot x + \frac{1}{2} \cdot x^{2} + \frac{-1}{3} \cdot x^{3} + \frac{1}{4} \cdot x^{4} + \frac{-1}{5} \cdot x^{5} + \frac{1}{6} \cdot x^{6} + \frac{-1}{7} \cdot x^{7} + \frac{1}{8} \cdot x^{8} + \frac{-1}{9} \cdot x^{9} + ...$


  0.010868 seconds (183.58 k allocations: 5.141 MiB)


In [31]:
SeriesFix{Rational{Int}}(s -> integrate(s, 1//1))

$1 + x + \frac{1}{2} \cdot x^{2} + \frac{1}{6} \cdot x^{3} + \frac{1}{24} \cdot x^{4} + \frac{1}{120} \cdot x^{5} + \frac{1}{720} \cdot x^{6} + \frac{1}{5040} \cdot x^{7} + \frac{1}{40320} \cdot x^{8} + \frac{1}{362880} \cdot x^{9} + ...$


In [32]:
SeriesFix{Rational{Int}}(s -> integrate(-integrate(s), 1//1))

$1 + 0 \cdot x + \frac{-1}{2} \cdot x^{2} + 0 \cdot x^{3} + \frac{1}{24} \cdot x^{4} + 0 \cdot x^{5} + \frac{-1}{720} \cdot x^{6} + 0 \cdot x^{7} + \frac{1}{40320} \cdot x^{8} + 0 \cdot x^{9} + ...$


In [33]:
SeriesFix{Rational{Int}}(s -> integrate(integrate(-s, 1//1)))

$0 + x + 0 \cdot x^{2} + \frac{-1}{6} \cdot x^{3} + 0 \cdot x^{4} + \frac{1}{120} \cdot x^{5} + 0 \cdot x^{6} + \frac{-1}{5040} \cdot x^{7} + 0 \cdot x^{8} + \frac{1}{362880} \cdot x^{9} + ...$
